In [1]:
import PublicDataReader as pdr
import pandas as pd
print(pdr.__version__)

serviceKey = 'Arjt7EQRVOdUPjLvpxVjFg9RvfEbWJb%2Bd5qNQ1h%2B9Q6nF37OgOVZNqGNZy4pW8XiYeBmrSeq%2BYUOi1N%2Bn2kgYg%3D%3D'

molit = pdr.Transaction(serviceKey)


>>> PublicDataReader Version : 2021.4.12

- Author : Wooil Jeong
- E-mail : wooil@kakao.com
- Github : https://github.com/WooilJeong/PublicDataReader
- Blog : https://wooiljeong.github.io

>>> 아파트매매 실거래자료 조회 서비스가 정상 작동합니다.
>>> 아파트매매 실거래 상세 자료 조회 서비스가 정상 작동합니다.
>>> 아파트 전월세 자료 조회 서비스가 정상 작동합니다.
>>> 아파트 분양권전매 신고 자료 조회 서비스가 정상 작동합니다.
>>> 오피스텔 매매 신고 조회 서비스가 정상 작동합니다.
>>> 오피스텔 전월세 신고 조회 서비스가 정상 작동합니다.
>>> 연립다세대 매매 실거래자료 조회 서비스가 정상 작동합니다.
>>> 연립다세대 전월세 실거래자료 조회 서비스가 정상 작동합니다.
>>> 단독/다가구 매매 실거래 조회 서비스가 정상 작동합니다.
>>> 단독/다가구 전월세 자료 조회 서비스가 정상 작동합니다.
>>> 토지 매매 신고 조회 서비스가 정상 작동합니다.
>>> 상업업무용 부동산 매매 신고 자료 조회 서비스가 정상 작동합니다.


In [2]:
def get_district_code(bdongName):
    district_name = bdongName
    codeResult = molit.CodeFinder(district_name)
    district_code = codeResult.iloc[0,1]
    return district_code

In [3]:
gu_name = ['강남구','강동구','강서구','강북구','관악구','광진구','구로구','금천구',
          '노원구','동대문구','도봉구','동작구','마포구','서대문구','성동구','성북구',
          '서초구','송파구','영등포구','용산구','양천구','은평구','종로구','중구','중랑구']

many = len(gu_name)

In [4]:
# 구 별로 dataframe 만들기
gu_df = []

for i in range(25):
    district_code = get_district_code(gu_name[i])
    #추가할 때 날짜만 바꿔서 사용
    gu_data = molit.DataCollector(molit.AptTrade, district_code, 202101, 202106)
    
    gu_df.append(gu_data)

>>> LAWD_CD : 11680 DEAL_YMD : 202101
>>> LAWD_CD : 11680 DEAL_YMD : 202102
>>> LAWD_CD : 11680 DEAL_YMD : 202103
>>> LAWD_CD : 11680 DEAL_YMD : 202104
>>> LAWD_CD : 11680 DEAL_YMD : 202105
>>> LAWD_CD : 11680 DEAL_YMD : 202106
>>> LAWD_CD : 11740 DEAL_YMD : 202101
>>> LAWD_CD : 11740 DEAL_YMD : 202102
>>> LAWD_CD : 11740 DEAL_YMD : 202103
>>> LAWD_CD : 11740 DEAL_YMD : 202104
>>> LAWD_CD : 11740 DEAL_YMD : 202105
>>> LAWD_CD : 11740 DEAL_YMD : 202106
>>> LAWD_CD : 11500 DEAL_YMD : 202101
>>> LAWD_CD : 11500 DEAL_YMD : 202102
>>> LAWD_CD : 11500 DEAL_YMD : 202103
>>> LAWD_CD : 11500 DEAL_YMD : 202104
>>> LAWD_CD : 11500 DEAL_YMD : 202105
>>> LAWD_CD : 11500 DEAL_YMD : 202106
>>> LAWD_CD : 11305 DEAL_YMD : 202101
>>> LAWD_CD : 11305 DEAL_YMD : 202102
>>> LAWD_CD : 11305 DEAL_YMD : 202103
>>> LAWD_CD : 11305 DEAL_YMD : 202104
>>> LAWD_CD : 11305 DEAL_YMD : 202105
>>> LAWD_CD : 11305 DEAL_YMD : 202106
>>> LAWD_CD : 11620 DEAL_YMD : 202101
>>> LAWD_CD : 11620 DEAL_YMD : 202102
>>> LAWD_CD 

In [23]:
import datetime

start_date = datetime.date(2006,1,1)

for i in range(25):
    gu_df[i]['면적당가격'] = gu_df[i]['거래금액']/gu_df[i]['전용면적']
    gu_df[i]['거래일'] = gu_df[i]['거래일'].astype(str)
    gu_df[i]['거래주'] = [(datetime.date((int)(date[:4]),(int)(date[5:7]),(int)(date[8:10]))-start_date).days//7 for date in gu_df[i]['거래일']]
    gu_df[i]['count'] = 1
    
    df = pd.read_pickle(f'../seoul_apt_trade_data/{gu_name[i]}_trade.pkl')
    
    df = df[df['거래일'] <= '2020-12-31'] 
    df = pd.concat([df,gu_df[i]], ignore_index = True)
    df.to_pickle(f'../seoul_apt_trade_data/{gu_name[i]}_trade.pkl')